In [1]:
%load_ext autotime

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import unicodedata
import nltk
import codecs
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

time: 5.6 s


In [3]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.neighbors import DistanceMetric
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

time: 57.7 ms


In [4]:
df = pd.read_pickle('./datasets/news_data.pickle')

time: 377 ms


In [5]:
df.sample(5)

,URL,CATEGORY,content and summary
91513,http://www.investing.com/news/stock-market-new...,business,[Japan Pharma Takeda To Fight $6B US Jury Dama...
60987,http://www.iol.co.za/business/international/us...,business,[US consumers step up spending\n\n\n\n\n\n\n\n...
58673,http://www.democratandchronicle.com/article/20...,entertainment,[]
72463,http://www.bucksfreepress.co.uk/leisure/showbi...,entertainment,[Lady Gaga has celebrated her 28th birthday wi...
97317,http://androidcommunity.com/t-mobile-covering-...,technology,"[As we mentioned earlier in the day, the Galax..."


time: 15.1 ms


In [6]:
df = df.dropna()
df.shape

(108043, 3)

time: 48.9 ms


In [7]:
df = df.loc[:,['CATEGORY','content and summary']]

time: 19.9 ms


In [8]:
df = df[df['content and summary'].map(lambda d: len(d)) > 0]

time: 85.3 ms


In [9]:
df.shape

(37033, 2)

time: 11 ms


In [10]:
def get_content(row):
    row = row[0]
    return row
def get_summary(row):
    row = row[1]
    return row

time: 1.98 ms


In [11]:

df['content'] = df['content and summary'].apply(get_content)
df['summary'] = df['content and summary'].apply(get_summary)

time: 46.9 ms


In [12]:
df.head()

,CATEGORY,content and summary,content,summary
50890,business,[The Federal Reserve approved Ally Financial I...,The Federal Reserve approved Ally Financial In...,The Federal Reserve approved Ally Financial In...
50898,business,[— Major shareholders of Duke Energy Corp. hav...,— Major shareholders of Duke Energy Corp. have...,— Major shareholders of Duke Energy Corp. have...
50900,business,[Photos taken earlier this month show that Nor...,Photos taken earlier this month show that Nort...,Photos taken earlier this month show that Nort...
50903,business,[Thanks to dogged reporting by the Associated ...,Thanks to dogged reporting by the Associated P...,Thanks to dogged reporting by the Associated P...
50906,business,[The energy giant says it is committed to clea...,The energy giant says it is committed to clean...,The energy giant says it is committed to clean...


time: 16.9 ms


In [13]:
df.drop(['content and summary','summary'],axis=1, inplace=True)


time: 57.8 ms


In [14]:
df.head()

,CATEGORY,content
50890,business,The Federal Reserve approved Ally Financial In...
50898,business,— Major shareholders of Duke Energy Corp. have...
50900,business,Photos taken earlier this month show that Nort...
50903,business,Thanks to dogged reporting by the Associated P...
50906,business,The energy giant says it is committed to clean...


time: 5.97 ms


In [15]:
df = df.rename(columns= {'CATEGORY':'category'})

time: 5.98 ms


In [16]:
df.head()

,category,content
50890,business,The Federal Reserve approved Ally Financial In...
50898,business,— Major shareholders of Duke Energy Corp. have...
50900,business,Photos taken earlier this month show that Nort...
50903,business,Thanks to dogged reporting by the Associated P...
50906,business,The energy giant says it is committed to clean...


time: 14 ms


#### Before cleaning the text

In [17]:
for i in range(1):
    print("Content")
    print("----------------------")
    print(df.content.iloc[i])
#     print("Summary")
#     print("----------------------")
#     print(df.summary.iloc[i])

Content
----------------------
The Federal Reserve approved Ally Financial Inc.’s capital plan in the bank regulator’s annual review of the industry’s financial health, clearing another potential hurdle to the auto lender’s plans to exit government ownership.

Ally’s plan was approved after the Federal Reserve found that the bank could keep lending in a severe economic downturn, according to a report Wednesday.
time: 2.99 ms


In [18]:
def clean_text(text):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Add contractions
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    
    text =  ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    
    
    
    wordnet_lemmatizer = WordNetLemmatizer()

    def get_tag(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return ''

    text_result = []
    tokens = word_tokenize(text)  # Generate list of tokens
    tagged = pos_tag(tokens)
    for t in tagged:
        try:
            text_result.append(wordnet_lemmatizer.lemmatize(t[0], get_tag(t[1][:2])))
        except:
            text_result.append(wordnet_lemmatizer.lemmatize(t[0]))
    paragraph = " ".join(str(x) for x in text_result)
    return paragraph

    

time: 7.98 ms


In [19]:
df['content'] = df['content'].apply(clean_text)
df.to_pickle('cleaned_data.pickle')

time: 9min 33s


In [4]:
df = pd.read_pickle('cleaned_data.pickle')

time: 238 ms


#### After cleaning the text

In [5]:
for i in range(1):
    print("Content ")
    print("----------------------")
    print(df.content.iloc[i])

Content 
----------------------
federal reserve approve ally financial inc ’ s capital plan bank regulator ’ s annual review industry ’ s financial health clearing another potential hurdle auto lender ’ s plan exit government ownership ally ’ s plan approve federal reserve find bank could keep lend severe economic downturn accord report wednesday
time: 2 ms


In [6]:
df.head()

,category,content
50890,0,federal reserve approve ally financial inc ’ s...
50898,0,— major shareholder duke energy corp call comp...
50900,0,photo take earlier month show north carolina r...
50903,0,thanks dog report associated press know active...
50906,0,energy giant say committed clean dan river spi...


time: 14 ms


#### Encoding the categories

In [7]:
label_encoder = LabelEncoder() 

df['category']= label_encoder.fit_transform(df['category'])

time: 3.99 ms


In [8]:
df.sample(5)

,category,content
169985,2,saudi arabia health ministry say sunday eight ...
92867,3,cnn name logo associate element ® © 2019 cable...
66411,0,consumer confidence rise first time since earl...
195303,2,thunder bay – mental health awareness week mha...
70260,0,guardian london ever widen financial lexicon l...


time: 8.98 ms


In [9]:
def knn(p,n,vector_type, metric=None):
        
    X_train, X_test, y_train, y_test = train_test_split(df["content"], df["category"],
                                                        test_size=0.2, random_state=5)
    
    knn = neighbors.KNeighborsClassifier(n_neighbors=n, weights='distance', algorithm='auto', metric=metric, p=p, n_jobs=1)
    
    if vector_type == 'BoW':
        vectorizer = CountVectorizer(analyzer='word', input='content')
        train_vec = vectorizer.fit_transform(X_train)
        test_vec = vectorizer.transform(X_test)
    
    elif vector_type == 'tfidf':
        vectorizer =  TfidfVectorizer(analyzer='word', input='content')
        train_vec = vectorizer.fit_transform(X_train)
        test_vec = vectorizer.transform(X_test)

    

    knn.fit(train_vec, y_train)
    predicted = knn.predict(test_vec)
    acc = metrics.accuracy_score(y_test, predicted)
    print('Accuracy: KNN with {}, neighbors: {}, distance metric: {}  = {}'.format(vector_type, n,metric, str(acc * 100) + '%'))


time: 5.98 ms


In [11]:
for n_neighbor in range(2,8,1):
    knn(2,n_neighbor,'BoW','cosine')
    

Accuracy: KNN with BoW, neighbors: 2, distance metric: cosine  = 94.00567031186715%
Accuracy: KNN with BoW, neighbors: 3, distance metric: cosine  = 94.14067773727555%
Accuracy: KNN with BoW, neighbors: 4, distance metric: cosine  = 94.43769407317403%
Accuracy: KNN with BoW, neighbors: 5, distance metric: cosine  = 94.11367625219387%
Accuracy: KNN with BoW, neighbors: 6, distance metric: cosine  = 94.30268664776563%
Accuracy: KNN with BoW, neighbors: 7, distance metric: cosine  = 94.12717699473471%
time: 5min 39s


In [10]:
for n_neighbor in range(2,8,1):
    knn(2,n_neighbor,'BoW','euclidean')

Accuracy: KNN with BoW, neighbors: 2, distance metric: euclidean  = 88.74038072093965%
Accuracy: KNN with BoW, neighbors: 3, distance metric: euclidean  = 87.2958012690698%
Accuracy: KNN with BoW, neighbors: 4, distance metric: euclidean  = 87.82233022816254%
Accuracy: KNN with BoW, neighbors: 5, distance metric: euclidean  = 87.91683542594842%
Accuracy: KNN with BoW, neighbors: 6, distance metric: euclidean  = 87.82233022816254%
Accuracy: KNN with BoW, neighbors: 7, distance metric: euclidean  = 87.67382206021331%
time: 5min 31s


In [10]:
for n_neighbor in range(2,8,1):
    knn(2,n_neighbor,'tfidf','cosine')

Accuracy: KNN with tfidf, neighbors: 2, distance metric: cosine  = 93.87066288645876%
Accuracy: KNN with tfidf, neighbors: 3, distance metric: cosine  = 94.35668961792899%
Accuracy: KNN with tfidf, neighbors: 4, distance metric: cosine  = 94.58620224112326%
Accuracy: KNN with tfidf, neighbors: 5, distance metric: cosine  = 94.5997029836641%
Accuracy: KNN with tfidf, neighbors: 6, distance metric: cosine  = 94.50519778587822%
Accuracy: KNN with tfidf, neighbors: 7, distance metric: cosine  = 94.23518293506142%
time: 5min 34s


In [11]:
for n_neighbor in range(2,8,1):
    knn(2,n_neighbor,'tfidf','euclidean')

Accuracy: KNN with tfidf, neighbors: 2, distance metric: euclidean  = 93.87066288645876%
Accuracy: KNN with tfidf, neighbors: 3, distance metric: euclidean  = 94.35668961792899%
Accuracy: KNN with tfidf, neighbors: 4, distance metric: euclidean  = 94.51869852841907%
Accuracy: KNN with tfidf, neighbors: 5, distance metric: euclidean  = 94.51869852841907%
Accuracy: KNN with tfidf, neighbors: 6, distance metric: euclidean  = 94.45119481571487%
Accuracy: KNN with tfidf, neighbors: 7, distance metric: euclidean  = 94.22168219252059%
time: 5min 14s


**Observation:** The best accuracy 94.599% is achieved with TF-IDF, num of neighbors : 5 and cosine similarity as the distance metric